<a href="https://colab.research.google.com/github/cit08/Phyton_2/blob/main/Copia_de_practica3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3
* Puede resolver aquí los ejercicios abriendo una celda después de cada uno de los enunciados.  
* Por favor, documente la solución de los ejercicios añadiendo suficientes **comentarios** del programador que expliquen el código.

## Ejercicio 3.1. Trabajar con un DataSet
* Obtener un dataset, preferiblemente con datos financieros, actuariales, económicos o estadísticos.
* Crear un modelo de análisis de los datos, utilizando las librerías principales para el análisis de datos, por ejemplo:
    - NumPy
    - Pandas
    - Matplotlib

## Ejercicio 3.2. Gráfico histórico de un índice o valor bursátil
* Descargar un archivo con formato ***csv*** de la página de Yahoo Finanzas con la cotización de:
 - un índice (IBEX, NASDAQ, ...) o
 - valor bursátil,
* o bien, de otra fuente disponible.
* Crear un procedimiento que lea el fichero y finalmente obtenga un gráfico del histórico del índice o valor.
* Tomar el periodo histórico más largo posible.
* Alternativamente o adicionalmente, si disponemos de una API o fuente de información diferente de un *csv* descargado de forma manual, también se podría realizar de una forma más automatizada.

## Ejercicio 3.3. Trabajar con la Librería openpyxl
* Crear una aplicación programada en Python que permita interactuar con la **hoja de cálculo** tanto para lectura, como para escritura.
* Utilice la Librería openpyxl, u otra que considere adecuada para interactuar con hoja de cálculo

## Ejercicio 3.4. Librería actuarial
* Utilizando la librería actuarial **pyliferisk**, o cualquier otra que considere adecuada en este campo, cree un caso donde se muestren diferentes cálculos y procedimientos que se puedan realizar con ella.
* Alternativamente, puede realizar su propia librería actuarial y probarla creando ejemplos prácticos. En este caso, no olvide documentar la funcionalidad de los métodos que programe, utilizando POO.

In [6]:
#!/usr/bin/python
from pyliferisk import MortalityTable
from pyliferisk.mortalitytables import SPAININE2004, GKM95

tariff = MortalityTable(nt=SPAININE2004)
experience = MortalityTable(nt=GKM95, perc=85)

# Print the omega (limiting age) of the both tables:
print(tariff.w)
print(experience.w)

# Print the qx at 50 years old:
print(tariff.qx[50] / 1000)
print(experience.qx[50] / 1000)

101
121
0.003113
0.003662395


In [5]:
!pip install pyliferisk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyliferisk: filename=pyliferisk-1.11-py3-none-any.whl size=20285 sha256=af556d028e77b3dc4ce738ba7080e5195d5778c9c533e07b71aee875d73766c0
  Stored in directory: /root/.cache/pip/wheels/9e/45/24/c5df681f6ecbf5e9a538e8c861a5ad8fbfac7fba88102d632d
Successfully built pyliferisk


In [7]:
from pyliferisk import Actuarial, Ax
from pyliferisk.mortalitytables import GKM95

mt = Actuarial(nt=GKM95, i=0.02)
x = 50 		#age
C = 1000 	#capital

print(Ax(mt, x) * C)

583.2765526571227


In [8]:
#!/usr/bin/python
from pyliferisk import *
from pyliferisk.mortalitytables import SPAIN_PASEM2010M
import pandas as pd

mt = MortalityTable(nt=SPAIN_PASEM2010M)

age = 67				
init_payment = 8000
incr = 0.03		# increment annutie
i = 0.05 		# interest rate

period = len(range(0, mt.w - age))

df = pd.DataFrame(pd.date_range('1/1/2016', periods=period, freq='Y'), columns=['date'])
df['t'] = df.index
df['age'] = pd.Series(list(range(age, mt.w)))
df['px'] = df['t'].apply(lambda t: 1 - mt.qx[age+t] / 1000)
df['px_cumprod'] = df['px'].cumprod()
df['disc_factor'] = df['t'].apply(lambda t: 1 / (1 + i) ** (t + 1))
df['capital'] = df['t'].apply(lambda t: init_payment * ((1 + incr) ** t))
df['payments'] = df['t'].apply(lambda t: init_payment if t == 0 else
		df['capital'][t] * df['px_cumprod'][t-1])
df['apv_payments'] = df['payments'] * df['disc_factor']
premium = df['apv_payments'].sum().round(2)

print(df.head())
print('Premium:', premium)

        date  t  age        px  px_cumprod  disc_factor     capital  \
0 2016-12-31  0   67  0.984336    0.984336     0.952381  8000.00000   
1 2017-12-31  1   68  0.982438    0.967049     0.907029  8240.00000   
2 2018-12-31  2   69  0.980193    0.947895     0.863838  8487.20000   
3 2019-12-31  3   70  0.977540    0.926605     0.822702  8741.81600   
4 2020-12-31  4   71  0.974395    0.902879     0.783526  9004.07048   

      payments  apv_payments  
0  8000.000000   7619.047619  
1  8110.928640   7356.851374  
2  8207.539047   7089.980820  
3  8286.321490   6817.177197  
4  8343.217031   6537.128856  
Premium: 96718.36


In [13]:
import pandas as pd
from pyliferisk import MortalityTable
import matplotlib.pyplot as plt

USLife1979 = pd.read_csv('uslife79.csv')
USLife79lx = USLife1979['lx'].tolist()

mt = MortalityTable(lx=USLife79lx)

fig= plt.figure(figsize=(7.5,3.7))

x = range(0, mt.w)
y = mt.lx[:mt.w]
plt.plot(x, y, linewidth=3, color='0')
plt.ylim(bottom=0)
plt.xlim(left=0)
plt.ylabel('lx')
plt.xlabel('age')
plt.show()

FileNotFoundError: ignored

In [10]:


#!/usr/bin/python
from pyliferisk import *
from pyliferisk.mortalitytables import INM05
import numpy as np
import pandas as pd

rfr = pd.read_excel('EIOPA_RFR_20161231_Term_Structures.xlsx', sheet_name='RFR_spot_no_VA', 
					skiprows=9, usecols='C:C', names=['Euro'])

tariff  = Actuarial(nt=INM05, i=0.05)
reserve = MortalityTable(nt=INM05)
x = 32			# age
Cd = 3000 		# capital death
Premium = Cd * Ax(tariff, 25) / annuity(tariff, 25, 'w', 0) #fixed at age 25

qx_vector = []
px_vector = []
for i in range(x,reserve.w + 1):
	qx = ((reserve.lx[i]-reserve.lx[i+1]) / reserve.lx[x])
	qx_vector.append(qx)
	qx_sum = sum(qx_vector)
	px_vector.append(1 - qx_sum)

def Reserve(i):
	discount_factor = []
	for y in range(0, reserve.w - x + 1):
		if isinstance(i,float):
			discount_factor.append(1 / (1 + i) ** (y + 1))
		elif i == 'rfr':
			discount_factor.append(1 / (1 + rfr['Euro'][y]) ** (y + 1))

	APV_Premium = np.dot(Premium, px_vector)
	APV_Claims = np.dot(Cd, qx_vector)
	return np.dot(discount_factor, np.subtract(APV_Claims, APV_Premium)).round(2)

print(Reserve(0.0191))
print(Reserve(0.0139))
print(Reserve('rfr'))

FileNotFoundError: ignored

## Ejercicio 3.5. Trabajar con una API
* Localizar una API con datos que puedan resultar de interés.
* Crear una aplicación programada en Python que utilice los datos de la API, preferiblemente con datos en tiempo real.
* Mostrar la información que nuestra aplicación pueda procesar, según ciertos filtros, que faciliten el uso de la API.

## Ejercicio 3.6. Página web con Flask
* Crear una aplicación de página web programada en Python con el framework Flask.
* Puede desplegarla en alguna plataforma gratuita como, por ejemplo, [Pythonanywhere](https://www.pythonanywhere.com) o [Heroku](https://www.heroku.com).

## Ejercicio 3.7. Bot de Telegram que proporciona cotizaciones en tiempo real
Crear un Bot de Telegram programado en Python que realizando **Web Scraping** nos pueda dar precios en tiempo real de valores cotizados en bolsa, o en mercados financieros internacionales.  
Adjuntar capturas de pantalla mostrando el funcionamiento del bot en Telegram.

## Ejercicio 3.8. Estrategias de Trading
* Elegir una compañía cotizada y realizar con ella diferentes estrategias de trading, con gráficos e indicadores bursátiles.
* Puede utilizar la librería TA-Lib u otro sistema para efectuar el análisis.

## Ejercicio 3.9. Finanzas
Estudie casos en el campo de las finanzas:
* VAN y TIR
* Cuadro de amortización de préstamos
* Operativa con activos de Renta Fija: bonos
* *Duration*
* Estructura Temporal de los Tipos de Interés (ETTI)
* Forwards sobre tipos de interés
* Arbitraje
* Random Walk en una y dos dimensiones
* ... / ...


## Ejercicio 3.10. Valoración de opciones financieras
* Método de valoración de opciones europeas mediante arbol binomial
* Método de Black-Scholes
* Convergencia de ambos métodos
* Gráfico de convergencia
* Arbitraje en caso de no cumplirse la paridad Put-Call